In [1]:
#.libPaths(c("/home/data/refdir/Rlib","/home/data/t060324/R/x86_64-pc-linux-gnu-library/4.1"))
library(scibet)
library(Seurat)
library(scater)
library(scran)
library(dplyr)
library(Matrix)
library(cowplot)
library(ggplot2)
library(harmony)
setwd('/home/data/t060324/jobs/aimin/')

Attaching SeuratObject

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, r

In [2]:
dir_ST <- './data/GSE211956_RAW/ST/'

In [3]:
sample <- list.files(dir_ST)
names(sample) <- substr(sample,1,10)
sample

GSM6506110               GSM6506111               GSM6506112 
"GSM6506110_SP1_spatial" "GSM6506111_SP2_spatial" "GSM6506112_SP3_spatial" 
              GSM6506113               GSM6506114               GSM6506115 
"GSM6506113_SP4_spatial" "GSM6506114_SP5_spatial" "GSM6506115_SP6_spatial" 
              GSM6506116               GSM6506117 
"GSM6506116_SP7_spatial" "GSM6506117_SP8_spatial"

In [4]:
for(sample_id in names(sample)){
files <- list.files(paste0(dir_ST,sample[sample_id],'/'))
mtx_file <- files[grepl(pattern = '_matrix.mtx.gz',files)]
feature_file <- files[grepl(pattern = 'features.tsv.gz',files)]
barcode_file <- files[grepl(pattern = 'barcodes.tsv.gz',files)]
exp <- readMM(file = paste0(dir_ST,sample[sample_id],'/',mtx_file))
barcode <- read.table(file = paste0(dir_ST,sample[sample_id],'/',barcode_file),header = F,sep = '\t')
features <- read.table(file = paste0(dir_ST,sample[sample_id],'/',feature_file),header = F,sep = '\t')
rownames(exp) <- features[,2]
colnames(exp) <- barcode[,1]
ST_obj <- Seurat::CreateSeuratObject(counts = exp, project = 'OV', assay = 'Spatial')
img <- Seurat::Read10X_Image(image.dir = paste0(dir_ST,sample[sample_id],'/spatial/'))
Seurat::DefaultAssay(object = img) <- 'Spatial'
img <- img[colnames(x = ST_obj)]
ST_obj[['image']] <- img
ST_obj <- SCTransform(ST_obj, assay = "Spatial",vars.to.regress = 'nCount_Spatial', verbose = FALSE)
ST_obj <- RunPCA(ST_obj, assay = "SCT", verbose = FALSE)
ST_obj <- FindNeighbors(ST_obj, reduction = "pca", dims = 1:30)
ST_obj <- RunUMAP(ST_obj, reduction = "pca", dims = 1:30)
ST_obj <- FindClusters(ST_obj,resolution = 1, verbose = FALSE) 
saveRDS(ST_obj,file = paste0('./data/GSE211956_RAW/',sample_id,'_ST.RDS'))
}



Warning message:
“Non-unique features (rownames) present in the input matrix, making unique”
Computing nearest neighbor graph

Computing SNN

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
21:20:01 UMAP embedding parameters a = 0.9922 b = 1.112

21:20:01 Read 2940 rows and found 30 numeric columns

21:20:01 Using Annoy for neighbor search, n_neighbors = 30

21:20:01 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

21:20:02 Writing NN index file to temp file /home/data/t060324/Rtmp/RtmplguSsi/file2ea0f87585c904

21:20:02 Searching Annoy

21:28:52 Initializing from normalized Laplacian + noise

21:28:52 Commencing optimization for 500 epochs, with 135602 positive edges

21:28:56 Optimization finished



In [ ]:
sample <- list.files(path = './data/GSE211956_RAW/',pattern = '_ST.RDS')
names(sample) <- substr(sample,1,10)
sample

In [18]:
ST_obj_list <- list()
for(sampleFile in names(sample)){
    ST_obj <- readRDS(file = paste0('./data/GSE211956_RAW/',sample[sampleFile]))
    print(sampleFile)
    ST_obj$sample <- sampleFile
    ST_obj$mitoRatio <- PercentageFeatureSet(object = ST_obj, pattern = "^MT-")
    ST_obj$mitoRatio <- ST_obj@meta.data$mitoRatio / 100
    ST_obj$log10GenesPerUMI <- log10(ST_obj$nFeature_Spatial)/log10(ST_obj$nCount_Spatial)
    ST_obj$riboRatio <- PercentageFeatureSet(object = ST_obj, pattern = "^RP[SL]")
    ST_obj$riboRatio <- ST_obj@meta.data$mitoRatio / 100
    ST_obj_list[[sampleFile]] <- ST_obj@meta.data
}

[1] "GSM6506110"
[1] "GSM6506111"
[1] "GSM6506112"
[1] "GSM6506113"
[1] "GSM6506114"
[1] "GSM6506115"
[1] "GSM6506116"
[1] "GSM6506117"


In [19]:
names(ST_obj_list)
names(sample)

[1] "GSM6506110" "GSM6506111" "GSM6506112" "GSM6506113" "GSM6506114"
[6] "GSM6506115" "GSM6506116" "GSM6506117"

[1] "GSM6506110" "GSM6506111" "GSM6506112" "GSM6506113" "GSM6506114"
[6] "GSM6506115" "GSM6506116" "GSM6506117"

In [20]:
ST_metaResponse <- data.frame(sample=c('GSM6506110','GSM6506111','GSM6506112','GSM6506113','GSM6506114','GSM6506115','GSM6506116','GSM6506117'),
                              response=c('poor','good','good','partial','good','partial','poor','poor')
                             )
rownames(ST_metaResponse) <- ST_metaResponse$sample

In [35]:
round(x = 1.03245,digits = 2)

[1] 1.03

In [45]:
median(ST_obj_list[[sampleFile]]$riboRatio,na.rm = T)

[1] 0.0006424274

In [41]:
ST_metaResponse$nSpots <- 0
ST_metaResponse$Median_nUMIs <- 0
ST_metaResponse$Median_nGenes <- 0
ST_metaResponse$mitoRatio <- 0
ST_metaResponse$riboRatio<- 0
for(sampleFile in names(sample)){
    print(sampleFile)
    ST_metaResponse[sampleFile,'Median_nUMIs'] <- paste0(round(median(ST_obj_list[[sampleFile]]$nCount_Spatial,na.rm = T),2),'(',round(quantile(ST_obj_list[[sampleFile]]$nCount_Spatial)[2],2),'-',round(quantile(ST_obj_list[[sampleFile]]$nCount_Spatial)[4],2),')')
    ST_metaResponse[sampleFile,'Median_nGenes'] <- paste0(round(median(ST_obj_list[[sampleFile]]$nFeature_Spatial,na.rm = T),2),'(',round(quantile(ST_obj_list[[sampleFile]]$nFeature_Spatial)[2],2),'-',round(quantile(ST_obj_list[[sampleFile]]$nFeature_Spatial)[4],2),')')
    ST_metaResponse[sampleFile,'riboRatio'] <- paste0(round(median(ST_obj_list[[sampleFile]]$riboRatio*100,na.rm = T),2),'(',round(quantile(ST_obj_list[[sampleFile]]$riboRatio*100)[2],2),'-',round(quantile(ST_obj_list[[sampleFile]]$riboRatio*100)[4],2),')')
    ST_metaResponse[sampleFile,'mitoRatio'] <- paste0(round(median(ST_obj_list[[sampleFile]]$mitoRatio*100,na.rm = T),2),'(',round(quantile(ST_obj_list[[sampleFile]]$mitoRatio*100)[2],2),'-',round(quantile(ST_obj_list[[sampleFile]]$mitoRatio*100)[4],2),')')
    ST_metaResponse[sampleFile,'nSpots'] <- nrow(ST_obj_list[[sampleFile]])
}

[1] "GSM6506110"
[1] "GSM6506111"
[1] "GSM6506112"
[1] "GSM6506113"
[1] "GSM6506114"
[1] "GSM6506115"
[1] "GSM6506116"
[1] "GSM6506117"


In [42]:
nrow(ST_obj_list[[sampleFile]])

[1] 3180

In [43]:
ST_metaResponse

,sample,response,nSpots,Median_nUMIs,Median_nGenes,mitoRatio,riboRatio
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
GSM6506110,GSM6506110,poor,2940,5723.5(3767.75-8688.5),2435.5(1771-3361.25),11.2(5.21-15.67),0.11(0.05-0.16)
GSM6506111,GSM6506111,good,1935,8377(4431-12540),3031(1897-4063.5),3.67(2.21-5.57),0.04(0.02-0.06)
GSM6506112,GSM6506112,good,1504,7194(4820.25-10479),3062(2282.5-3984.75),4.69(2.58-6.87),0.05(0.03-0.07)
GSM6506113,GSM6506113,partial,3007,6165(4418.5-8416.5),2542(1978.5-3296),10.3(6.41-14.1),0.1(0.06-0.14)
GSM6506114,GSM6506114,good,2124,5677(2687.75-13730.75),2343.5(1332-4114.5),3.64(2.23-5.08),0.04(0.02-0.05)
GSM6506115,GSM6506115,partial,1801,6241(3091-11837),2437(1453-3787),6(3.78-9.07),0.06(0.04-0.09)
GSM6506116,GSM6506116,poor,3632,4251.5(2061-9195.5),1915(1115-3258.25),6.33(4.16-7.78),0.06(0.04-0.08)
GSM6506117,GSM6506117,poor,3180,5030.5(2951-8121),2226.5(1531.5-3141.5),6.42(5.03-8.19),0.06(0.05-0.08)


In [46]:
write.table(x = ST_metaResponse,file = './output//bulk/section_6/table_ST.txt',sep = '\t',quote = F,row.names = F)